http://www.nltk.org/howto/corpus.html

In [91]:
import nltk
from nltk.corpus import reuters, stopwords
from xml.etree import ElementTree
import numpy as np
from utils import *

nltk.download('reuters')
nltk.download('punkt')


[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\monish\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\monish\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [96]:
dataset_shakespeare = []
with open("Shakespeare/Shakespeare_all_works", 'rb') as f:
    lines = f.read().decode('utf-8').split('\n')
    for line in lines:
        if(line != ""):
            dataset_shakespeare.append(line.lower())
            
dataset_reuters = [reuters.paras(fileids=[i]) for i in reuters.fileids()]
print(dataset_shakespeare[0])

﻿


# PreProcessing

# Training

# Validation

# BRENT